# "Frueher War Alles Besser": Schneefall in Deutschland in Zeiten des Klimawandels

[fill in introductory text here]

In [18]:
# load needed libraries and packages  
import numpy as np
import matplotlib.pyplot as plt  
import pandas as pd 
import seaborn as sns
import datetime as datetime
%matplotlib inline  
#import bokeh as bokeh
from bokeh.charts import TimeSeries, gridplot
from bokeh.plotting import figure,show,output_file,save
from bokeh.io import output_notebook  
from bokeh.models import Range1d

In [15]:
pd.__version__

'0.18.1'

In [2]:
output_notebook()

Loading BokehJS ...

# Stationsdaten  

Zuerst muessen wir herausfinden, wo die Wetterstationen stehen und wie lange es ueberhaupt Daten gibt. 

In [3]:
# create a function that reads in the list of stations and returns a dataframe 
def read_station_metadata():

    # path to data  
    datadir='/home/lneef/Data/DataScienceStories/historical/'
    fstations='KL_Tageswerte_Beschreibung_Stationen.txt'

    # read in the station metadata as a pandas dataframe 
    colnames=['Stations_id','von_datum','bis_datum','Stationshoehe','geoBreite','geoLaenge','Stationsname','Bundesland']
    #dtypes = ['int64','str','str','float','float','float','str','str']
    stations = pd.read_csv(datadir+fstations,header=None,skiprows=2,sep='\s+', encoding='latin1',names=colnames,
                           dtype={'von_datum':'str','bis_datum':'str'})

    # Bundesland is a categorical variable. Everything else is either unique or continuous. 
    stations["Bundesland"] = stations["Bundesland"].astype('category')
    
    return(stations)

In [4]:
# now we can query all kinds of things. Like how many stationss are there for each bundesland?
S = read_station_metadata()
print(S.head(5))
S['Bundesland'].value_counts()


  Stations_id von_datum bis_datum  Stationshoehe  geoBreite  geoLaenge  \
0           1  19370101  19860630          478.0    47.8413     8.8493   
1           3  18910101  20110331          202.0    50.7827     6.0941   
2          44  19710301  20160224           44.0    52.9335     8.2370   
3          52  19730101  20011231           46.0    53.6623    10.1990   
4          61  19750701  19780831          339.0    48.8443    12.6171   

           Stationsname           Bundesland  
0                  Aach    Baden-Württemberg  
1                Aachen  Nordrhein-Westfalen  
2          Großenkneten        Niedersachsen  
3  Ahrensburg-Wulfsdorf   Schleswig-Holstein  
4            Aiterhofen               Bayern  


Bayern                    224
Baden-Württemberg         165
Niedersachsen             111
Nordrhein-Westfalen        87
Hessen                     84
Schleswig-Holstein         65
Thüringen                  55
Rheinland-Pfalz            55
Sachsen-Anhalt             53
Sachsen                    52
Brandenburg                42
Mecklenburg-Vorpommern     40
Berlin                     21
Saarland                   16
Hamburg                    12
Bremen                      4
Name: Bundesland, dtype: int64

In [169]:
all_stations = list(S['Stationsname'].values)
len(all_stations)

test=S.query('Bundesland=="Hessen"')
hessen_stations = list(test['Stationsname'].values)


['Alsfeld-Eifa', 'Alsfeld-Reibertenrod', 'Arolsen-Neu-Berich', 'Aßlar-Klein-Altenstädten', 'Beerfelden', 'Bensheim', 'Biedenkopf-Wallau', 'Brombachtal-Kirch-Brombach', 'Büdingen', 'Burgwald-Bottendorf', 'Darmstadt', 'Darmstadt(A)', 'Darmstadt-Botanischer-Garten', 'Dillenburg', 'Eschwege', 'Frankfurt/Main', 'Frankfurt/Main(Stadt)', 'Frankfurt/Main-Westend', 'Frankfurt/Main(Feldbergstr.)', 'Fritzlar(Flugplatz)', 'Fulda', 'Geisenheim', 'Gelnhausen', 'Gernsheim', 'Gießen(Lahntal)', 'Gießen/Wettenberg', 'Gilserberg-Moischeid', 'Grebenhain-Herchenhain', 'Gründau-Breitenborn', 'Hanau', 'Heidenrod-Mappershain', 'Herleshausen', 'Hersfeld,BAd', 'Hofgeismar-Beberbeck', 'Homberg/Ohm', 'Kassel', 'Kassel-Harleshausen', 'Kleiner-Feldberg/Taunus', 'Korbach-Lengefeld', 'Langen(BTZ)', 'Limburg/Lahn-Offheim', 'Lindenfels-Winterkasten', 'Löhnberg-Obershausen', 'Lorch/Rhein', 'Cölbe,KR.Marburg-Biedenkopf', 'Michelstadt', 'Michelstadt-Vielbrunn', 'Mittel-Gründau', 'Modautal-Neunkirchen', 'Nauheim,BAd', 'Neu

#  Aggregierung der Klimadaten

In [5]:
# define a function that, given a station name and a list of meteorolgical variables, 
# returns relevant data  
def station_data_to_dataframe(station_name,variable_list=None,debug=False):
    
    # find the station number corresponding to the station name   
    S = read_station_metadata()
    sid = S[S['Stationsname']==station_name]['Stations_id'].values[0].zfill(5)
    if debug:
        print(sid)
    
    # find the start and end dates of the data available for that station
    datestr0 = S[S['Stationsname']==station_name]['von_datum'].values[0]
    datestrf = S[S['Stationsname']==station_name]['bis_datum'].values[0]
    
    # here a complication is that we only have data up to 31 Dec 2015, 
    # even though the station summary has a bis_datum that goes farther. 
    # ...so correct for that here 
    yearf = int(datestrf[0:4])
    if yearf > 2015:
        datestrf = '20151231'
    
    # find the filename corresponding to this station number
    datadir='/home/lneef/Data/DataScienceStories/historical/'
    stationdir = 'tageswerte_'+sid+'_'+datestr0+'_'+datestrf+'_'+'hist/'
    ff = 'produkt_klima_Tageswerte_'+datestr0+'_'+datestrf+'_'+sid+'.txt'
    fname = datadir+stationdir+ff
    if debug:
        print(fname)
        
    # read this file into a pandas dataframe 
    try:
        DF = pd.read_csv(fname,sep=';',index_col=' MESS_DATUM') 
    except OSError:
        print("Unable to find data for station "+station_name)
        return
    
    # convert the index to a date 
    DF.index = pd.to_datetime(DF.index,format="%Y%m%d")
    DF.index.name='Date'

    # strip the whitespace out of column names and make lowercase 
    DF.rename(columns=lambda x: x.replace(" ", "").lower(), inplace=True)  
    
    # subset the variables that we want 
    if variable_list is None:
        DFv = DF
    else:
        DFv = DF.loc[:,variable_list]
    
    # only show winter values 
    # TODO: make this chooseable with an input to the function 
    select_dates = [dd for dd in DF.index if dd.month in [1,2,12]]
    DFwinter = DFv.ix[select_dates]

    # replace all -999 values with NaNs
    if variable_list is None:
        variable_list_orig=['QUALITAETS_NIVEAU', 'LUFTTEMPERATUR','DAMPFDRUCK','BEDECKUNGSGRAD'
                       ,'LUFTDRUCK_STATIONSHOEHE','REL_FEUCHTE','WINDGESCHWINDIGKEIT', 'LUFTTEMPERATUR_MAXIMUM',
                       'LUFTTEMPERATUR_MINIMUM','LUFTTEMP_AM_ERDB_MINIMUM', 'WINDSPITZE_MAXIMUM', 'NIEDERSCHLAGSHOEHE',
                       'NIEDERSCHLAGSHOEHE_IND','SONNENSCHEINDAUER', 'SCHNEEHOEHE']
        variable_list = [x.replace(" ","").lower() for x in variable_list_orig]
    variable_list = [v.replace(" ","").lower() for v in variable_list]
    for variable in variable_list:
        mask = DFwinter.loc[:,variable]<-990
        DFwinter.loc[:,variable][mask]=np.nan

    # smooth with an annual (winter) mean
    DFout = DFwinter.resample('AS').mean()
    
    # addd a column containing the station name string 
    DFout['Station']=station_name
    
    return(DFout)


In [26]:
# try it out for a sample station
D = station_data_to_dataframe('Aachen',['schneehoehe'])
#D = station_data_to_dataframe('Dillenburg')
D.head(5)

/home/lneef/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


,schneehoehe,Station
Date,,
1891-01-01,NaN,Aachen
1892-01-01,4.800000,Aachen
1893-01-01,7.277778,Aachen
1894-01-01,2.166667,Aachen
1895-01-01,15.594203,Aachen


# Visualization 1: Comparing stations  


In [11]:
# small wrapper function for creating nice timeseries plots 
def create_tseries_per_station(data,station):
    import bokeh as bokeh
    p= bokeh.charts.TimeSeries(data,
                                   title=station,
                                   xlabel='Date',
                                   ylabel='Schneehoehe (cm)',
                                   legend=False,

    )
    return p

In [13]:
# given a list of stations, retrieve the data for each station, and plot snowcover together

# retrieve the data 
station_list = ['Dillenburg','Kiel-Holtenau','Berlin-Tegel']
Dlist = [station_data_to_dataframe(station,['schneehoehe']) for station in station_list]
Dbig = pd.concat(Dlist)

# grid plot configuration 
plot_options = dict(plot_width=300, plot_height=300,
                   tools="pan,wheel_zoom,box_zoom,box_select",
                  )

# loop over stations and create plots 
plotlist = [create_tseries_per_station(D,station) for D,station in zip(Dlist,station_list)]

# give them all the same x- andd y-ranges
from bokeh.models import Range1d
for s in plotlist:
    s.y_range=Range1d(Dbig.schneehoehe.min(),Dbig.schneehoehe.max())
    s.x_range=Range1d(Dbig.index[0],Dbig.index[len(Dbig.index)-1])

# apply the plots to the grid and show it 
grid = gridplot(plotlist, ncols=3,**plot_options)
show(grid)



In [14]:
plotlist

## Comparing stations - comments & ideas  

+ it's hard for a layperson viewer to put in a list of stations 
+ would be cool if you could click on a peak and get a google image for that city and that winter 

# Visualization 2: Single bundesland average vs. individual cities 

This plot would give a more averaged view, but also show how single stations deviate from one another. 


In [6]:
# Function to plot the timeseries of one of the variables over a given Bundesland
def plot_timeseries_by_bundesland(Bundesland,variable,Nstations=None,debug=False):

    # bokeh extras
    from bokeh.models import Range1d,HoverTool,ColumnDataSource

    # read in station data as a dataframe 
    S = read_station_metadata()
    
    # find the stations that are in the requested bundesland
    stationdata_bundesland=S[S["Bundesland"]==Bundesland]
    # limit the number of stations, so that the figure isn't too big, by  
    # randomly choosing some stations 
    if Nstations is not None:
        Nsample = min(Nstations,len(stationdata_bundesland))
        stationdata_bundesland=stationdata_bundesland.sample(n=Nsample)
    station_list = list(stationdata_bundesland['Stationsname'].values)  
    if debug:
        print(station_list)

    # read in the data and create a big dataframe 
    for station in station_list:
        try:
            station_data_to_dataframe(station,[variable])
        except OSError:
            print(station)
    Dlist = [station_data_to_dataframe(station,[variable]) for station in station_list]
    Dbig = pd.concat(Dlist)
    data = Dbig.reset_index().pivot(index='Date',columns='Station',values=variable)
    if debug:
        print(data.head(5))
    
    hover = HoverTool(tooltips=[("Winter:", "@DateStr"),("Station:","@Station"),(variable+":","$y")])
    from palettable.wesanderson import GrandBudapest3_6 as bmap
    
    p = figure(
        plot_width=800, 
        plot_height=400, 
        x_axis_type="datetime")

    for station, df in Dbig.reset_index().groupby('Station'):
        source = ColumnDataSource(df[['Date', variable,'Station']])
        source.add(df.Date.map(lambda x: x.strftime('%Y')), 'DateStr')
        color = "blue" 
        p.line(x='Date', y=variable, color=bmap.hex_colors[1], legend=False,
               line_width=2, alpha=0.5,
               hover_color=bmap.hex_colors[2],hover_alpha=1.0,
               source=source)
    p.add_tools(hover)

    p.xaxis.axis_label = "Datum"
    p.yaxis.axis_label = variable
    p.legend.location = "top_left"
    p.title.text = Bundesland+' '+variable
    

    d0 = pd.tslib.Timestamp('1938-01-01 00:00:00')
    df = pd.tslib.Timestamp('2015-01-01 00:00:00')
    p.x_range=Range1d(d0,df)

    return(p)

In [9]:
p = plot_timeseries_by_bundesland('Hessen','lufttemperatur',Nstations=20,debug=False)
show(p)
type(p)

Unable to find data for station Weilburg(Kläranlage)
Unable to find data for station Weilburg(Kläranlage)


bokeh.plotting.figure.Figure

In [ ]:
# Now let's do a grid plot over all bundeslaender  
output_file("bundeslaender.html", title="Schneefall und Temperatur in Deutschen Bundeslaendern")

# list of bundeslaender 
S = read_station_metadata()
Blist = list(S['Bundesland'].cat.categories)

# now run each BL through the plotting function, once for snow and once for temp 
N=20
for B in Blist:
    try:
        test = plot_timeseries_by_bundesland(B,'schneehoehe',Nstations=N)
    except OSError:
        print(B)
        
        
plotlist1 = [plot_timeseries_by_bundesland(B,'schneehoehe',Nstations=N) for B in Blist]
plotlist2 = [plot_timeseries_by_bundesland(B,'lufttemperatur',Nstations=N) for B in Blist]

# grid plot configuration 
plot_options = dict(plot_width=400, plot_height=200
                  # tools="pan,wheel_zoom,box_zoom,box_select",
                  )

# put these two lists together such that the lists run vertically  
plotarr = np.array([plotlist1,plotlist2]).transpose()
PL = plotarr.tolist()

# give them all the same x- andd y-ranges
from bokeh.models import Range1d
# snow depth 
for s in plotlist1:
    s.y_range=Range1d(0,30)
# temp 
for s in plotlist2:
    s.y_range=Range1d(-6,8)

# create a grid and throw the created plots onto it 
grid=gridplot(children=PL,
        toolbar_location='right',
             **plot_options)


show(grid)

INFO:bokeh.core.state:Session output file 'bundeslaender.html' already exists, will be overwritten.


Unable to find data for station Tölz,BAd
Unable to find data for station Tölz,BAd
Unable to find data for station Berlin-Alexanderplatz
Unable to find data for station Berlin-Alexanderplatz


/home/lneef/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2881: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/lneef/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:12: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.


Unable to find data for station Sachsa,BAd
Unable to find data for station Sachsa,BAd
Unable to find data for station Bonn-Hardthöhe
Unable to find data for station Bonn-Hardthöhe


/home/lneef/anaconda3/lib/python3.5/site-packages/pandas/core/generic.py:4485: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/home/lneef/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/lneef/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Unable to find data for station Neuhausen/Erzgeb.-Rauschenbach(Talsp.)
Unable to find data for station Neuhausen/Erzgeb.-Rauschenbach(Talsp.)
Unable to find data for station Ingolstadt(Flussmeisterstelle)
Unable to find data for station Ingolstadt(Flussmeisterstelle)
Unable to find data for station Berlin-Alexanderplatz
Unable to find data for station Berlin-Alexanderplatz


/home/lneef/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:17: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.


Unable to find data for station Willingen/Upland
Unable to find data for station Willingen/Upland
Unable to find data for station Reichshof-Eckenhagen
Unable to find data for station Bonn-Hardthöhe
Unable to find data for station Reichshof-Eckenhagen
Unable to find data for station Bonn-Hardthöhe


In [ ]:
# The above plot is a lot to handle. Let's instead make individual plots for each bundesland
from bokeh.plotting import reset_output

# how many stations to show for each BL?
N=None

# list of bundeslaender 
S = read_station_metadata()
Blist = list(S['Bundesland'].cat.categories)

# list of variables to plot
variable_list=['lufttemperatur','schneehoehe']

for B in Blist:
    reset_output()
    print(B)

    # Now let's do a grid plot over all bundeslaender  
    output_file("single_row_"+B+".html", title="Temperatur und Schneefall in "+B)

    # now run each BL through the plotting function, once for snow and once for temp 
    PL = [plot_timeseries_by_bundesland(B,variable,Nstations=N) for variable in variable_list]

    # grid plot configuration 
    plot_options = dict(plot_width=800, plot_height=400,
                       tools="pan,wheel_zoom,box_zoom,box_select",
                      )

    grid = gridplot(PL, ncols=1,**plot_options)
    save(grid)

Baden-Württemberg
Unable to find data for station Baiersbronn-Obertal
Unable to find data for station Donaueschingen
Unable to find data for station Eberbach(LUBW)
Unable to find data for station Hinterzarten


/home/lneef/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:22: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.


Unable to find data for station Lenzkirch-Ruhbühl
Unable to find data for station Sankt-Blasien-Menzenschwand
Unable to find data for station Titisee-Neustadt-Langenordnach
Unable to find data for station Todtmoos
Unable to find data for station Baiersbronn-Obertal
Unable to find data for station Donaueschingen
Unable to find data for station Eberbach(LUBW)
Unable to find data for station Hinterzarten
Unable to find data for station Lenzkirch-Ruhbühl
Unable to find data for station Sankt-Blasien-Menzenschwand
Unable to find data for station Titisee-Neustadt-Langenordnach
Unable to find data for station Todtmoos
Unable to find data for station Baiersbronn-Obertal
Unable to find data for station Donaueschingen
Unable to find data for station Eberbach(LUBW)
Unable to find data for station Hinterzarten
Unable to find data for station Lenzkirch-Ruhbühl
Unable to find data for station Sankt-Blasien-Menzenschwand
Unable to find data for station Titisee-Neustadt-Langenordnach
Unable to find da